# Подключение библиотек

In [ ]:
#!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import catboost
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных

In [ ]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data_path = "/content/drive/MyDrive/intern_task.csv"
data = pd.read_csv(data_path)
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


In [ ]:
features = list(data.columns[1::])

Обработаем пропуски в данных

In [ ]:
data = data.dropna()

In [ ]:
print(max(data['rank']))

4


In [ ]:
sorted_df = data.sort_values(['query_id', 'rank'], ascending=[True, False])
sorted_df.tail(10)

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance
235212,0,29995,1.0,0.0,1.0,1.0,1.0,0.5,0.0,0.5,...,0.0,0.166667,0.471409,13.302685,0.5,0.007937,0.000051,16.0,8.0,1.0
235226,0,29995,1.0,0.0,1.0,1.0,1.0,0.5,0.0,0.5,...,0.0,0.100000,0.471409,26.605371,0.5,0.019608,0.000290,13.0,6.5,1.0
235231,0,29995,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,...,0.0,0.125000,0.471409,13.302685,0.5,0.013682,0.000165,41.0,20.5,1.0
235232,0,29995,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,...,3.2,0.142857,0.471409,13.302685,0.5,0.015284,0.000087,4.0,2.0,1.0
235233,0,29995,1.0,0.0,1.0,0.0,1.0,0.5,0.0,0.5,...,0.0,0.000000,0.471409,26.605371,0.0,0.011521,0.000109,13.0,6.5,1.0
235234,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,0.0,0.000000,0.471409,0.000000,0.0,0.001536,0.000002,2.0,1.0,1.0
235235,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,25.6,0.000000,0.471409,0.000000,0.0,0.000535,0.000000,1.0,0.5,1.0
235239,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,35.3,0.000000,0.471409,0.000000,0.0,0.000499,0.000000,1.0,0.5,1.0
235243,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,0.0,0.000000,0.471409,0.000000,0.0,0.004732,0.000023,6.0,3.0,1.0
235249,0,29995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.076923,0.000000,0.000000,0.5,0.006410,0.000000,0.0,0.0,1.0


Разделим данные на тренировочную и тестовую выборку

In [ ]:
train_data, val_data = train_test_split(data, test_size = 0.2)
print(f"Размер тренировочной выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(val_data)}")

Размер тренировочной выборки: 188206
Размер тестовой выборки: 47052


# Применение Catboost.Classifier

In [ ]:
train_features = train_data[features]
train_target = train_data['rank']
val_features = val_data[features]
val_target = val_data['rank']

In [ ]:
model = catboost.CatBoostClassifier()
model.fit(train_features, train_target)

Learning rate set to 0.103355
0:	learn: 1.4996448	total: 750ms	remaining: 12m 29s
1:	learn: 1.4189150	total: 1.42s	remaining: 11m 47s
2:	learn: 1.3544528	total: 2.34s	remaining: 12m 57s
3:	learn: 1.3024410	total: 3.53s	remaining: 14m 39s
4:	learn: 1.2590982	total: 4.96s	remaining: 16m 27s
5:	learn: 1.2228356	total: 6.11s	remaining: 16m 51s
6:	learn: 1.1923226	total: 6.87s	remaining: 16m 14s
7:	learn: 1.1667214	total: 7.58s	remaining: 15m 39s
8:	learn: 1.1444508	total: 8.54s	remaining: 15m 40s
9:	learn: 1.1257874	total: 9.22s	remaining: 15m 12s
10:	learn: 1.1097501	total: 9.98s	remaining: 14m 57s
11:	learn: 1.0958925	total: 10.7s	remaining: 14m 43s
12:	learn: 1.0835603	total: 11.4s	remaining: 14m 29s
13:	learn: 1.0723351	total: 12.1s	remaining: 14m 13s
14:	learn: 1.0625981	total: 12.8s	remaining: 14m 2s
15:	learn: 1.0540519	total: 13.4s	remaining: 13m 43s
16:	learn: 1.0465902	total: 14.1s	remaining: 13m 33s
17:	learn: 1.0394772	total: 14.8s	remaining: 13m 25s
18:	learn: 1.0337473	total:

In [ ]:
predictions = model.predict(val_features)
preds = np.array([predictions[i][0] for i in range(len(predictions))])
print(preds)
print(val_target.to_numpy())
ndcg = ndcg_score([val_target.to_numpy()], [preds], k=5)
print(f"NDCG_5: {ndcg}")

[1 0 0 ... 0 0 2]
[2 1 0 ... 0 0 1]
NDCG_5: 0.7574626865671641


# Применение Catboost.Regressor

В предыдущем пункте наше решение основывалось на полном доверии к классификатору, теперь рассмторим другой подход

Будем действовать по следующему алгортму:

1) Расставим релевантность докумета запросу. Переведя "категориальные" позиции из {0, 1, 2, 3, 4} в удобные для регрессии вещественные из [0, 1] так, чтобы при максимальной "релевантности" получали минимальную позицию и наоборот

2) Применим ```Regressor``` $\rightarrow$ получим числа соответствующие "релевантоности" документа запросу, то есть чем полученное число больше, тем документ подходит запросу качесвтеннее

3) Разобьем документы на группы согласно полученному значению релевантности (сделаем это равномерно для пяти позиций)

Таким образом, получим модель ранжирования, чем выше "релевантность", тем лучше документ подходит запросу

Соотнесем ```rank``` и релевантность

In [44]:
rank_to_relevance = {0: 1, 1: 0.75, 2: 0.5, 3: 0.25, 4: 0}
data['relevance'] = data['rank'].map(rank_to_relevance)
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333,1.00
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667,0.75
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333,1.00
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667,0.75
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000,0.50


Применим ```Regressor```

In [46]:
train_data, val_data = train_test_split(data, test_size = 0.2)

train_features = train_data[features]
train_target = train_data['relevance']
val_features = val_data[features]
val_target = val_data['relevance']

model = catboost.CatBoostRegressor()
model.fit(train_features, train_target)

Learning rate set to 0.093664
0:	learn: 0.2058514	total: 147ms	remaining: 2m 27s
1:	learn: 0.2040774	total: 270ms	remaining: 2m 14s
2:	learn: 0.2025734	total: 392ms	remaining: 2m 10s
3:	learn: 0.2013050	total: 527ms	remaining: 2m 11s
4:	learn: 0.2001480	total: 661ms	remaining: 2m 11s
5:	learn: 0.1991675	total: 808ms	remaining: 2m 13s
6:	learn: 0.1983497	total: 935ms	remaining: 2m 12s
7:	learn: 0.1975849	total: 1.06s	remaining: 2m 11s
8:	learn: 0.1969691	total: 1.18s	remaining: 2m 9s
9:	learn: 0.1964052	total: 1.32s	remaining: 2m 10s
10:	learn: 0.1958888	total: 1.44s	remaining: 2m 9s
11:	learn: 0.1954184	total: 1.56s	remaining: 2m 8s
12:	learn: 0.1949751	total: 1.71s	remaining: 2m 9s
13:	learn: 0.1945826	total: 1.86s	remaining: 2m 11s
14:	learn: 0.1942716	total: 1.98s	remaining: 2m 9s
15:	learn: 0.1939377	total: 2.11s	remaining: 2m 10s
16:	learn: 0.1936186	total: 2.24s	remaining: 2m 9s
17:	learn: 0.1933289	total: 2.37s	remaining: 2m 9s
18:	learn: 0.1930660	total: 2.51s	remaining: 2m 9s


Теперь запишем каждому документу предсказание модели

In [49]:
predictions = model.predict(val_features)
val_data['predictions'] = predictions
val_data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance,predictions
149549,2,18580,2.0,0.0,2.0,0.0,2.0,1.000000,0.000000,1.000000,...,0.000000,0.966713,10.243247,0.0,0.036891,0.000874,54.0,27.000000,0.50,0.789156
53089,1,7435,3.0,2.0,3.0,3.0,3.0,1.000000,0.666667,1.000000,...,0.250000,0.960613,23.298098,1.0,0.017644,0.000018,29.0,9.666667,0.75,0.735281
144185,0,18040,3.0,0.0,0.0,0.0,3.0,1.000000,0.000000,0.000000,...,0.000000,0.980481,0.000000,0.0,0.010612,0.000006,17.0,5.666667,1.00,0.896252
115230,1,14710,1.0,0.0,1.0,1.0,1.0,0.500000,0.000000,0.500000,...,0.166667,0.119823,7.085005,0.5,0.030000,0.000625,9.0,4.500000,0.75,0.787514
118012,0,14965,1.0,0.0,1.0,0.0,1.0,0.333333,0.000000,0.333333,...,0.000000,0.548727,7.252768,0.0,0.008000,0.000113,11.0,3.666667,1.00,0.897390


In [53]:
val_data['pred_rank'] = val_data['predictions'].apply(lambda x: 0 if x >= 0.8 else (1 if 0.6 <= x < 0.8 else (2 if 0.4 <= x < 0.6 else (3 if 0.2 <= x < 0.4 else 4))))
val_data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance,predictions,pred_rank
149549,2,18580,2.0,0.0,2.0,0.0,2.0,1.000000,0.000000,1.000000,...,0.966713,10.243247,0.0,0.036891,0.000874,54.0,27.000000,0.50,0.789156,1
53089,1,7435,3.0,2.0,3.0,3.0,3.0,1.000000,0.666667,1.000000,...,0.960613,23.298098,1.0,0.017644,0.000018,29.0,9.666667,0.75,0.735281,1
144185,0,18040,3.0,0.0,0.0,0.0,3.0,1.000000,0.000000,0.000000,...,0.980481,0.000000,0.0,0.010612,0.000006,17.0,5.666667,1.00,0.896252,0
115230,1,14710,1.0,0.0,1.0,1.0,1.0,0.500000,0.000000,0.500000,...,0.119823,7.085005,0.5,0.030000,0.000625,9.0,4.500000,0.75,0.787514,1
118012,0,14965,1.0,0.0,1.0,0.0,1.0,0.333333,0.000000,0.333333,...,0.548727,7.252768,0.0,0.008000,0.000113,11.0,3.666667,1.00,0.897390,0


Считаем метрику

In [54]:
ndcg = ndcg_score([val_data['rank']], [val_data['pred_rank']], k=5)
print(f"NDCG_5: {ndcg}")

NDCG_5: 0.7894736842105263


# Итог

Таким образом, нами было предложено два метода по ранжированию документов:

**1)** Использование catboost.Classifier, полученное значение метрики на валиационной выборке составило **0.76**

**2)** Метод, использующий регрессию, основанный на получении "релевантностей" документов, на валидационной выборке значение метрики получилось **0.79**

Лучшее решение - второй метод, **финальное качество: 0.79**